In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns=100

In [94]:
headers=pd.read_csv('../hwy/bcm_iter1/headers.csv')
headers['headers']=headers['headers'].apply(lambda x: x.replace("'",''))
header_list= headers['headers'].unique()

In [142]:

load_am = pd.read_csv('../hwy/bcm_iter1/LOADAM.csv', names=header_list)
load_pm = pd.read_csv('../hwy/bcm_iter1/LOADPM.csv', names=header_list)
load_md = pd.read_csv('../hwy/bcm_iter1/LOADMD.csv', names=header_list)
load_ea = pd.read_csv('../hwy/bcm_iter1/LOADEA.csv', names=header_list)
load_ev = pd.read_csv('../hwy/bcm_iter1/LOADEV.csv', names=header_list)

In [150]:
columns=['A',
        'B','MODEL_LINK_','FT','AT','COUNTY','DISTANCE','CAP_HRLN','V_1','FFS','LANES','USECLASS','TOLLBOOTH',
        'VC_1',
        'CSPD_1',
        'VDT_1',
        'VT_1','V1_1','V2_1','V3_1', 'V4_1','V5_1','V6_1','V7_1','V8_1','V9_1','V10_1']
bcm_ft_dict={1:'Freeway GP',2:'Expressway',3:'Ramp',4:'Arterial', 5:'Arterial',6:'Collector', 7:'Local', 8:'CentroidConnector'}
def custom_fac_type(df):
    toll = df['TOLLBOOTH']
    fac_type=df['FT']
    useclass=df['USECLASS']
    if toll in range(1,12):
        return 'bridge toll'
    elif toll==0:
        if (fac_type==1):
            if useclass in [0,4]:
                return 'Freeway GP'
            else:
                return 'Freeway HOV'
        elif (fac_type==2):
            return 'Expressway'
        else:
            return 'Others'
    else:
        return 'Expressway'
    

load_am=load_am[columns]
load_am['TIMEPERIOD'] = 'AM'
load_pm=load_pm[columns]
load_pm['TIMEPERIOD'] = 'PM'
load_md=load_md[columns]
load_md['TIMEPERIOD'] = 'MD'
load_ea=load_ea[columns]
load_ea['TIMEPERIOD'] = 'EA'
load_ev=load_ev[columns]
load_ev['TIMEPERIOD'] = 'EV'

def update(df):

    df=df.rename(columns={'MODEL_LINK_':'MODEL_LINK_ID'})
    df['FAC_TYPE']=df.apply(custom_fac_type, axis=1)
    df['COUNTY']=df['COUNTY'].apply(lambda x: x.replace("'",''))
    df['CAPACITY_TOT']=df['CAP_HRLN']*df['LANES']
    df['LANE_MILES']=df['LANES']*df['DISTANCE']
    df['DA_VOL']=df['V1_1']+df['V6_1']
    df['SR2_VOL']=df['V2_1']+df['V7_1']
    df['SR3_VOL']=df['V3_1']+df['V8_1']
    df['VSM_VOL']=df['V4_1']+df['V9_1']
    df['LRG_TRK_VOL']=df['V5_1']+df['V10_1']
    df['TOT_MODEL_VOL']=df['DA_VOL']+df['SR2_VOL']+df['SR3_VOL']+df['VSM_VOL']+df['LRG_TRK_VOL']
    return df[['A','B','MODEL_LINK_ID','FT','FAC_TYPE','TIMEPERIOD','AT','COUNTY','DISTANCE','CAP_HRLN','V_1','FFS','LANES','USECLASS','TOLLBOOTH',
            'VC_1','CSPD_1', 'VDT_1', 'CAPACITY_TOT', 'LANE_MILES', 
            'DA_VOL', 'SR2_VOL', 'SR3_VOL',
            'VSM_VOL', 'LRG_TRK_VOL','TOT_MODEL_VOL']]


combined_loaded_network = pd.concat([update(load_am), 
                                     update(load_pm), 
                                     update(load_md), 
                                     update(load_ea), 
                                     update(load_ev)], ignore_index=True)


combined_loaded_network.head()

,A,B,MODEL_LINK_ID,FT,FAC_TYPE,TIMEPERIOD,AT,COUNTY,DISTANCE,CAP_HRLN,V_1,FFS,LANES,USECLASS,TOLLBOOTH,VC_1,CSPD_1,VDT_1,CAPACITY_TOT,LANE_MILES,DA_VOL,SR2_VOL,SR3_VOL,VSM_VOL,LRG_TRK_VOL,TOT_MODEL_VOL
0,1,1001671,77108,8,Others,AM,2,San Francisco,0.17232,700,194.10550,18,1,0,0,0.06932,18.0,33.44738,700,0.17232,27.13785,3.29607,2.80059,122.44585,19.21257,174.89293
1,1,1021297,78851,8,Others,AM,2,San Francisco,0.08112,700,237.53448,18,1,0,0,0.08483,18.0,19.26855,700,0.08112,133.01215,4.30353,4.04941,60.38429,17.89255,219.64193
2,1,1024255,79119,8,Others,AM,2,San Francisco,0.07073,700,90.21001,18,1,0,0,0.03222,18.0,6.38091,700,0.07073,0.00000,0.05040,0.00000,71.42986,9.36488,80.84514
3,2,1006730,77532,8,Others,AM,1,San Francisco,0.12418,2050,427.75143,18,1,0,0,0.05216,18.0,53.11959,2050,0.12418,148.23254,6.92378,38.49382,180.29900,26.90113,400.85027
4,2,1012189,78081,8,Others,AM,1,San Francisco,0.07230,2050,462.70731,18,1,0,0,0.05643,18.0,33.45312,2050,0.07230,157.54846,226.73677,14.51695,41.16844,11.36834,451.33896


In [152]:
at_dict={1:'CBD', 2:'Urban Business', 3:'Urban', 4:'Suburban', 5:'Rural', 0:'Regional Core'}

# combined_loaded_network['facilityType'] = combined_loaded_network.FT.map(bcm_ft_dict)
# combined_loaded_network['facilityType']=np.where(combined_loaded_network['USECLASS'] in [2,3], 
#                                                 'Freeway HOV', 
#                                                 combined_loaded_network['facilityType'])
combined_loaded_network['areaType'] = combined_loaded_network.AT.map(at_dict)


In [154]:
combined_loaded_network_24_hours = combined_loaded_network.groupby(['A', 
                                                                    'B', 'MODEL_LINK_ID',
                                                                    'FT', 
                                                                    'AT', 
                                                                    'COUNTY',
                                                                    'DISTANCE', 
                                                                    'FAC_TYPE', 
                                                                    'areaType']).agg({'CAP_HRLN':'mean',
                                                                                      'V_1':'sum',
                                                                                      'FFS':'mean',
                                                                                      'LANES':'mean',
                                                                                      'VC_1':'mean',
                                                                                      'CSPD_1':'mean',
                                                                                      'VDT_1':'sum',
                                                                                      'CAPACITY_TOT':'mean',
                                                                                      'LANE_MILES':'sum',
                                                                                      'DA_VOL':'sum',
                                                                                      'SR2_VOL':'sum',
                                                                                      'SR3_VOL':'sum',
                                                                                      'VSM_VOL':'sum',
                                                                                      'LRG_TRK_VOL':'sum',
                                                                                      'TOT_MODEL_VOL':'sum'}).reset_index()
combined_loaded_network_24_hours['TIMEPERIOD']='24 Hour'
combined_loaded_network_all_tp=pd.concat([combined_loaded_network, combined_loaded_network_24_hours], ignore_index=True)
combined_loaded_network_all_tp.head()

,A,B,MODEL_LINK_ID,FT,FAC_TYPE,TIMEPERIOD,AT,COUNTY,DISTANCE,CAP_HRLN,V_1,FFS,LANES,USECLASS,TOLLBOOTH,VC_1,CSPD_1,VDT_1,CAPACITY_TOT,LANE_MILES,DA_VOL,SR2_VOL,SR3_VOL,VSM_VOL,LRG_TRK_VOL,TOT_MODEL_VOL,areaType
0,1,1001671,77108,8,Others,AM,2,San Francisco,0.17232,700.0,194.10550,18.0,1.0,0.0,0.0,0.06932,18.0,33.44738,700.0,0.17232,27.13785,3.29607,2.80059,122.44585,19.21257,174.89293,Urban Business
1,1,1021297,78851,8,Others,AM,2,San Francisco,0.08112,700.0,237.53448,18.0,1.0,0.0,0.0,0.08483,18.0,19.26855,700.0,0.08112,133.01215,4.30353,4.04941,60.38429,17.89255,219.64193,Urban Business
2,1,1024255,79119,8,Others,AM,2,San Francisco,0.07073,700.0,90.21001,18.0,1.0,0.0,0.0,0.03222,18.0,6.38091,700.0,0.07073,0.00000,0.05040,0.00000,71.42986,9.36488,80.84514,Urban Business
3,2,1006730,77532,8,Others,AM,1,San Francisco,0.12418,2050.0,427.75143,18.0,1.0,0.0,0.0,0.05216,18.0,53.11959,2050.0,0.12418,148.23254,6.92378,38.49382,180.29900,26.90113,400.85027,CBD
4,2,1012189,78081,8,Others,AM,1,San Francisco,0.07230,2050.0,462.70731,18.0,1.0,0.0,0.0,0.05643,18.0,33.45312,2050.0,0.07230,157.54846,226.73677,14.51695,41.16844,11.36834,451.33896,CBD


In [163]:
combined_loaded_network_all_tp['DA_VMT']=combined_loaded_network_all_tp['DA_VOL']*combined_loaded_network_all_tp['DISTANCE']
combined_loaded_network_all_tp['SR2_VMT']=combined_loaded_network_all_tp['SR2_VOL']*combined_loaded_network_all_tp['DISTANCE']
combined_loaded_network_all_tp['SR3_VMT']=combined_loaded_network_all_tp['SR3_VOL']*combined_loaded_network_all_tp['DISTANCE']
combined_loaded_network_all_tp['VSM_VMT']=combined_loaded_network_all_tp['VSM_VOL']*combined_loaded_network_all_tp['DISTANCE']
combined_loaded_network_all_tp['LRG_TRK_VMT']=combined_loaded_network_all_tp['LRG_TRK_VOL']*combined_loaded_network_all_tp['DISTANCE']

combined_loaded_network_all_tp['TOTAL_MODEL_VMT']=combined_loaded_network_all_tp['TOT_MODEL_VOL']*combined_loaded_network_all_tp['DISTANCE']
combined_loaded_network_all_tp.head()


,A,B,MODEL_LINK_ID,FT,FAC_TYPE,TIMEPERIOD,AT,COUNTY,DISTANCE,CAP_HRLN,V_1,FFS,LANES,USECLASS,TOLLBOOTH,VC_1,CSPD_1,VDT_1,CAPACITY_TOT,LANE_MILES,DA_VOL,SR2_VOL,SR3_VOL,VSM_VOL,LRG_TRK_VOL,TOT_MODEL_VOL,areaType,DA_VMT,SR2_VMT,SR3_VMT,VSM_VMT,LRG_TRK_VMT,TOTAL_MODEL_VMT
0,1,1001671,77108,8,Others,AM,2,San Francisco,0.17232,700.0,194.10550,18.0,1.0,0.0,0.0,0.06932,18.0,33.44738,700.0,0.17232,27.13785,3.29607,2.80059,122.44585,19.21257,174.89293,Urban Business,4.676394,0.567979,0.482598,21.099869,3.310710,30.137550
1,1,1021297,78851,8,Others,AM,2,San Francisco,0.08112,700.0,237.53448,18.0,1.0,0.0,0.0,0.08483,18.0,19.26855,700.0,0.08112,133.01215,4.30353,4.04941,60.38429,17.89255,219.64193,Urban Business,10.789946,0.349102,0.328488,4.898374,1.451444,17.817353
2,1,1024255,79119,8,Others,AM,2,San Francisco,0.07073,700.0,90.21001,18.0,1.0,0.0,0.0,0.03222,18.0,6.38091,700.0,0.07073,0.00000,0.05040,0.00000,71.42986,9.36488,80.84514,Urban Business,0.000000,0.003565,0.000000,5.052234,0.662378,5.718177
3,2,1006730,77532,8,Others,AM,1,San Francisco,0.12418,2050.0,427.75143,18.0,1.0,0.0,0.0,0.05216,18.0,53.11959,2050.0,0.12418,148.23254,6.92378,38.49382,180.29900,26.90113,400.85027,CBD,18.407517,0.859795,4.780163,22.389530,3.340582,49.777587
4,2,1012189,78081,8,Others,AM,1,San Francisco,0.07230,2050.0,462.70731,18.0,1.0,0.0,0.0,0.05643,18.0,33.45312,2050.0,0.07230,157.54846,226.73677,14.51695,41.16844,11.36834,451.33896,CBD,11.390754,16.393068,1.049575,2.976478,0.821931,32.631807


In [204]:
county_dict=['San Francisco',
             'San Mateo',
             'Santa Clara',
             'Alameda',
             'Contra Costa',
             'Solano',
             'Napa',
             'Sonoma',
             'Marin',
             'San Joaquin']

county_sorter_index = dict(zip(county_dict, range(len(county_dict))))

ft_sorter=['Freeway GP', 'Freeway HOV','Expressway', 'bridge toll', 'Others']

ft_sorter_index = dict(zip(ft_sorter, range(len(ft_sorter))))

bcm_fac_summary={}
bcm_county_summary={}

bcm_fac_summary_vmt={}
bcm_county_summary_vmt={}
bcm_county_fac_summary_vmt={}
for time in ['AM','24 Hour']:
    _df=combined_loaded_network_all_tp[combined_loaded_network_all_tp['TIMEPERIOD']==time]
    bcm_summary_fac_type=_df.groupby(['FAC_TYPE']).agg( NumberOfRecords=('MODEL_LINK_ID','count'),
                                                        Total_Lane_Miles=('LANE_MILES','sum'),
                                                        Mean_Hourly_Capacity=('CAPACITY_TOT','mean'),
                                                        Mean_Congested_Speed=('CSPD_1','mean'),
                                                        Mean_FFS=('FFS','mean'),
                                                        Mean_Volume=('V_1','mean'),
                                                        TOT_VMT=('VDT_1','sum')).sort_index(key=lambda x: x.map(ft_sorter_index)).reset_index()
    bcm_summary_county=_df.groupby(['COUNTY']).agg( NumberOfRecords=('MODEL_LINK_ID','count'),
                                                    Total_Lane_Miles=('LANE_MILES','sum'),
                                                    Mean_Hourly_Capacity=('CAPACITY_TOT','mean'),
                                                    Mean_Congested_Speed=('CSPD_1','mean'),
                                                    Mean_FFS=('FFS','mean'),
                                                    Mean_Volume=('V_1','mean'),
                                                    TOT_VMT=('VDT_1','sum')).sort_index(key=lambda x: x.map(county_sorter_index)).reset_index()
    bcm_summary_fac_type_vmt=_df.groupby(['FAC_TYPE'])[ 'DA_VMT',
                                                        'SR2_VMT', 
                                                        'SR3_VMT', 
                                                        'VSM_VMT', 
                                                        'LRG_TRK_VMT',
                                                        'TOTAL_MODEL_VMT'].sum().sort_index(key=lambda x: x.map(ft_sorter_index)).reset_index()

    bcm_summary_county_vmt=_df.groupby(['COUNTY'])[ 'DA_VMT',
                                                    'SR2_VMT', 
                                                    'SR3_VMT', 
                                                    'VSM_VMT', 
                                                    'LRG_TRK_VMT',
                                                    'TOTAL_MODEL_VMT'].sum().sort_index(key=lambda x: x.map(county_sorter_index)).reset_index()

    bcm_summary_county_fac_vmt=_df.groupby(['COUNTY','FAC_TYPE'])[ 'DA_VMT',
                                                    'SR2_VMT', 
                                                    'SR3_VMT', 
                                                    'VSM_VMT', 
                                                    'LRG_TRK_VMT',
                                                    'TOTAL_MODEL_VMT'].sum().sort_index(key=lambda x: x.map(county_sorter_index)).reset_index()

    bcm_summary_fac_type['TIMEPERIOD']=time
    bcm_summary_county['TIMEPERIOD']=time
    bcm_summary_fac_type_vmt['TIMEPERIOD']=time
    bcm_summary_county_vmt['TIMEPERIOD']=time
    bcm_summary_county_fac_vmt['TIMEPERIOD']=time
    bcm_fac_summary[time] = bcm_summary_fac_type
    bcm_county_summary[time] = bcm_summary_county
    bcm_fac_summary_vmt[time] = bcm_summary_fac_type_vmt
    bcm_county_summary_vmt[time] = bcm_summary_county_vmt
    bcm_county_fac_summary_vmt[time] = bcm_summary_county_fac_vmt

bcm_all_fac=pd.concat(bcm_fac_summary.values(), ignore_index=True)[['FAC_TYPE', 'TIMEPERIOD', 'NumberOfRecords', 'Total_Lane_Miles','Mean_Hourly_Capacity', 'Mean_Congested_Speed', 'Mean_FFS','Mean_Volume', 'TOT_VMT']]
bcm_all_county=pd.concat(bcm_county_summary.values(), ignore_index=True)[['COUNTY', 'TIMEPERIOD', 'NumberOfRecords', 'Total_Lane_Miles','Mean_Hourly_Capacity', 'Mean_Congested_Speed', 'Mean_FFS','Mean_Volume', 'TOT_VMT']]
bcm_all_fac_vmt=pd.concat(bcm_fac_summary_vmt.values(), ignore_index=True)[['FAC_TYPE', 'TIMEPERIOD', 'DA_VMT', 'SR2_VMT', 'SR3_VMT', 'VSM_VMT', 'LRG_TRK_VMT', 'TOTAL_MODEL_VMT']]
bcm_all_county_vmt=pd.concat(bcm_county_summary_vmt.values(), ignore_index=True)[['COUNTY', 'TIMEPERIOD', 'DA_VMT', 'SR2_VMT', 'SR3_VMT', 'VSM_VMT', 'LRG_TRK_VMT', 'TOTAL_MODEL_VMT']]
bcm_all_county_fac_vmt=pd.concat(bcm_county_fac_summary_vmt.values(), ignore_index=True)[['COUNTY','FAC_TYPE', 'TIMEPERIOD', 'DA_VMT', 'SR2_VMT', 'SR3_VMT', 'VSM_VMT', 'LRG_TRK_VMT', 'TOTAL_MODEL_VMT']]
                                                        

C:\Users\USJH70~1\AppData\Local\Temp/ipykernel_20368/1592871344.py:40: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bcm_summary_fac_type_vmt=_df.groupby(['FAC_TYPE'])[ 'DA_VMT',
C:\Users\USJH70~1\AppData\Local\Temp/ipykernel_20368/1592871344.py:47: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bcm_summary_county_vmt=_df.groupby(['COUNTY'])[ 'DA_VMT',
C:\Users\USJH70~1\AppData\Local\Temp/ipykernel_20368/1592871344.py:54: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bcm_summary_county_fac_vmt=_df.groupby(['COUNTY','FAC_TYPE'])[ 'DA_VMT',


In [205]:
bcm_all_county_fac_vmt.head()

,COUNTY,FAC_TYPE,TIMEPERIOD,DA_VMT,SR2_VMT,SR3_VMT,VSM_VMT,LRG_TRK_VMT,TOTAL_MODEL_VMT
0,San Francisco,Expressway,AM,4.937316e+04,11120.605421,4134.757203,5509.569115,1217.187906,7.135528e+04
1,San Francisco,Freeway GP,AM,4.898574e+05,86760.305318,33994.546454,33803.873063,7850.289808,6.522664e+05
2,San Francisco,Others,AM,7.516916e+05,175479.841352,61323.683177,96761.226210,17918.716280,1.103175e+06
3,San Mateo,Expressway,AM,1.088217e+05,17287.015591,6340.435800,7330.169756,1499.960342,1.412793e+05
4,San Mateo,Freeway GP,AM,3.256271e+06,411626.815598,105481.560685,197695.911348,53643.702975,4.024719e+06


In [172]:
import xlsxwriter
import os

In [203]:
set_output_dir='../validation/Output/'

writer = pd.ExcelWriter(os.path.join(set_output_dir,'VMT_Summary_Comparison_Update.xlsx'), engine='xlsxwriter')


bcm_all_county.to_excel(writer, sheet_name='County Volumes', startrow=1, header=False, index=False)

workbook  = writer.book
worksheet = writer.sheets['County Volumes']
# Add a header format.
header_format = workbook.add_format({'bold': True,
                                    'text_wrap': True,
                                    'valign': 'top',
                                    'fg_color': '#D7E4BC',
                                    'border': 1})
number_format = workbook.add_format({'num_format': '_(* #,##0_);_(* (#,##0);_(* "-"??_);_(@_)'})

# Get the dimensions of the dataframe.
(max_row, max_col) = bcm_all_county.shape

# Make the columns wider for clarity.
worksheet.set_column(0,  max_col - 1, 18)
worksheet.autofilter(0, 0, max_row, max_col - 1)


for col_num, value in enumerate(bcm_all_county.columns.values):
    worksheet.write(0, col_num, value, header_format)

worksheet.set_column(2, max_col, 14, number_format)


bcm_all_fac.to_excel(writer, sheet_name='Fac Type Volumes', startrow=1, header=False, index=False)

workbook  = writer.book
worksheet = writer.sheets['Fac Type Volumes']
# Add a header format.
header_format = workbook.add_format({'bold': True,
                                    'text_wrap': True,
                                    'valign': 'top',
                                    'fg_color': '#D7E4BC',
                                    'border': 1})
number_format = workbook.add_format({'num_format': '_(* #,##0_);_(* (#,##0);_(* "-"??_);_(@_)'})

# Get the dimensions of the dataframe.
(max_row, max_col) = bcm_all_fac.shape

# Make the columns wider for clarity.
worksheet.set_column(0,  max_col - 1, 18)
worksheet.autofilter(0, 0, max_row, max_col - 1)


for col_num, value in enumerate(bcm_all_fac.columns.values):
    worksheet.write(0, col_num, value, header_format)

worksheet.set_column(2, max_col, 16, number_format)



bcm_all_county_vmt.to_excel(writer, sheet_name='County VMT', startrow=1, header=False, index=False)

workbook  = writer.book
worksheet = writer.sheets['County VMT']
# Add a header format.
header_format = workbook.add_format({'bold': True,
                                    'text_wrap': True,
                                    'valign': 'top',
                                    'fg_color': '#D7E4BC',
                                    'border': 1})
number_format = workbook.add_format({'num_format': '_(* #,##0_);_(* (#,##0);_(* "-"??_);_(@_)'})

# Get the dimensions of the dataframe.
(max_row, max_col) = bcm_all_county_vmt.shape

# Make the columns wider for clarity.
worksheet.set_column(0,  max_col - 1, 18)
worksheet.autofilter(0, 0, max_row, max_col - 1)


for col_num, value in enumerate(bcm_all_county_vmt.columns.values):
    worksheet.write(0, col_num, value, header_format)

worksheet.set_column(2, max_col, 14, number_format)


bcm_all_fac_vmt.to_excel(writer, sheet_name='Fac Type VMT', startrow=1, header=False, index=False)

workbook  = writer.book
worksheet = writer.sheets['Fac Type VMT']
# Add a header format.
header_format = workbook.add_format({'bold': True,
                                    'text_wrap': True,
                                    'valign': 'top',
                                    'fg_color': '#D7E4BC',
                                    'border': 1})
number_format = workbook.add_format({'num_format': '_(* #,##0_);_(* (#,##0);_(* "-"??_);_(@_)'})

# Get the dimensions of the dataframe.
(max_row, max_col) = bcm_all_fac_vmt.shape

# Make the columns wider for clarity.
worksheet.set_column(0,  max_col - 1, 14)
worksheet.autofilter(0, 0, max_row, max_col - 1)


for col_num, value in enumerate(bcm_all_fac_vmt.columns.values):
    worksheet.write(0, col_num, value, header_format)

worksheet.set_column(2, max_col, 16, number_format)



bcm_all_county_fac_vmt.to_excel(writer, sheet_name='County Fac Type VMT', startrow=1, header=False, index=False)

workbook  = writer.book
worksheet = writer.sheets['County Fac Type VMT']
# Add a header format.
header_format = workbook.add_format({'bold': True,
                                    'text_wrap': True,
                                    'valign': 'top',
                                    'fg_color': '#D7E4BC',
                                    'border': 1})
number_format = workbook.add_format({'num_format': '_(* #,##0_);_(* (#,##0);_(* "-"??_);_(@_)'})

# Get the dimensions of the dataframe.
(max_row, max_col) = bcm_all_fac_vmt.shape

# Make the columns wider for clarity.
worksheet.set_column(0,  max_col - 1, 14)
worksheet.autofilter(0, 0, max_row, max_col - 1)


for col_num, value in enumerate(bcm_all_fac_vmt.columns.values):
    worksheet.write(0, col_num, value, header_format)

worksheet.set_column(3, max_col, 16, number_format)

writer.save()

In [192]:
list(range(1, max_col+1))

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [161]:
mtc_pems=pd.read_csv('../validation/Input/mtc_pems_count_2015.csv')

mtc_pems_by_tp=mtc_pems[mtc_pems['type']=='ML'].groupby(['link_id','time_period']).sum().reset_index()

mtc_pems_all_tp=mtc_pems[mtc_pems['type']=='ML'].groupby(['link_id'])['count'].sum().reset_index()
mtc_pems_all_tp['time_period']='24 Hour'
mtc_pems_all=pd.concat([mtc_pems_by_tp, mtc_pems_all_tp], ignore_index=True)[['link_id','time_period','count']]
mtc_pems_all.tail()

,link_id,time_period,count
5450,8031439.0,24 Hour,56985.332685
5451,8033670.0,24 Hour,84191.453214
5452,8504042.0,24 Hour,20136.719907
5453,8518375.0,24 Hour,17134.457431
5454,8550768.0,24 Hour,53152.019618


In [66]:
combined_loaded_network_all_tp.facilityType.unique()

array(['CentroidConnector', 'Local', 'Arterial', 'Collector', nan, 'Ramp',
       'Freeway GP', 'Expressway', 'Freeway HOV'], dtype=object)

In [67]:
comparison = pd.merge(combined_loaded_network_all_tp, 
                      mtc_pems_all, 
                      left_on=['MODEL_LINK_ID','TIMEPERIOD'], 
                      right_on=['link_id','time_period'])
comparison['MODEL_VOL']=comparison['DA_VOL']+comparison['SR2_VOL']+comparison['SR3_VOL']+comparison['VSM_VOL']+comparison['LRG_TRK_VOL']
comparison['ERROR'] = comparison['MODEL_VOL'] - comparison['count']
comparison['SQUARED_ERROR'] =comparison['ERROR']**2

comparison=comparison[comparison['facilityType'].isin(['Freeway GP', 'Expressway', 'Freeway HOV'])]
comparison=comparison.sort_values(by=['MODEL_LINK_ID','TIMEPERIOD'])

comparison.head(10)

,A,B,MODEL_LINK_ID,FT,AT,COUNTY,DISTANCE,CAP_HRLN,V_1,FFS,LANES,USECLASS,VC_1,CSPD_1,VDT_1,VT_1,V1_1,V2_1,V3_1,V4_1,V5_1,V6_1,V7_1,V8_1,V9_1,V10_1,TIMEPERIOD,CAPACITY_TOT,LANE_MILES,DA_VOL,SR2_VOL,SR3_VOL,VSM_VOL,LRG_TRK_VOL,facilityType,areaType,link_id,time_period,count,MODEL_VOL,ERROR,SQUARED_ERROR
4530,1002422,1021187,2440,1,3,San Francisco,0.59638,2100.0,53470.01026,60.0,5.0,NaN,0.207834,59.800606,31888.33821,53470.01026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24 Hour,10500.0,14.90950,38126.11992,6438.46434,1985.68976,5817.61583,147.08366,Freeway GP,Urban,2440.0,24 Hour,78546.673041,52514.97351,-26031.699531,6.776494e+08
0,1002422,1021187,2440,1,3,San Francisco,0.59638,2100.0,6395.85059,60.0,5.0,0.0,0.152280,59.999160,3814.34473,6395.85059,5210.14258,809.09875,80.05337,165.56534,31.26463,0.0000,0.0,0.0,0.00000,0.0,AM,10500.0,2.98190,5210.14258,809.09875,80.05337,165.56534,31.26463,Freeway GP,Urban,2440.0,AM,17365.450704,6296.12467,-11069.326034,1.225300e+08
2725,1002422,1021187,2440,1,3,San Francisco,0.59638,2100.0,1216.06787,60.0,5.0,0.0,0.035100,60.000000,725.23615,1216.06787,910.27000,56.97000,1.15000,159.17635,23.77000,0.0000,0.0,0.0,0.00000,0.0,EA,10500.0,2.98190,910.27000,56.97000,1.15000,159.17635,23.77000,Freeway GP,Urban,2440.0,EA,3094.287879,1151.33635,-1942.951529,3.775061e+06
3632,1002422,1021187,2440,1,3,San Francisco,0.59638,2100.0,11942.68750,60.0,5.0,0.0,0.129250,59.999690,7122.35596,11942.68750,8134.55000,1927.79000,653.37384,917.35632,14.52000,0.0000,0.0,0.0,0.00000,0.0,EV,10500.0,2.98190,8134.55000,1927.79000,653.37384,917.35632,14.52000,Freeway GP,Urban,2440.0,EV,16559.266667,11647.59016,-4911.676507,2.412457e+07
1819,1002422,1021187,2440,1,3,San Francisco,0.59638,2100.0,13085.94141,60.0,5.0,0.0,0.226600,59.990880,7804.16797,13085.94141,7307.07422,1764.04285,406.11383,3412.46118,36.68213,0.0000,0.0,0.0,0.00000,0.0,MD,10500.0,2.98190,7307.07422,1764.04285,406.11383,3412.46118,36.68213,Freeway GP,Urban,2440.0,MD,19104.344262,12926.37421,-6177.970052,3.816731e+07
909,1002422,1021187,2440,1,3,San Francisco,0.59638,2100.0,20829.46289,60.0,5.0,0.0,0.495940,59.013300,12422.23340,20829.46289,16563.60742,1880.56274,844.99872,1162.96814,40.84690,0.4757,0.0,0.0,0.08850,0.0,PM,10500.0,2.98190,16564.08312,1880.56274,844.99872,1163.05664,40.84690,Freeway GP,Urban,2440.0,PM,22423.323529,20493.54812,-1929.775409,3.724033e+06
4531,1004082,1015322,4204,1,2,San Francisco,0.27655,2100.0,82356.04590,60.0,5.0,NaN,0.344590,57.975988,22775.33044,82356.04590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24 Hour,10500.0,6.91375,53936.82367,9681.29307,3468.40773,12490.26382,1293.95560,Freeway GP,Urban Business,4204.0,24 Hour,120995.805313,80870.74389,-40125.061423,1.610021e+09
1,1004082,1015322,4204,1,2,San Francisco,0.27655,2100.0,31379.18555,60.0,5.0,0.0,0.747120,50.190670,8677.82422,31379.18555,24094.40234,4543.36230,1355.21606,596.42218,340.05115,0.0000,0.0,0.0,0.25783,0.0,AM,10500.0,1.38275,24094.40234,4543.36230,1355.21606,596.68001,340.05115,Freeway GP,Urban Business,4204.0,AM,30832.685714,30929.71186,97.026146,9.414073e+03
2726,1004082,1015322,4204,1,2,San Francisco,0.27655,2100.0,6090.47559,60.0,5.0,0.0,0.175770,59.998010,1684.30383,6090.47559,5087.35449,398.27000,93.84923,210.61635,150.19284,0.0000,0.0,0.0,0.00000,0.0,EA,10500.0,1.38275,5087.35449,398.27000,93.84923,210.61635,150.19284,Freeway GP,Urban Business,4204.0,EA,12784.890625,5940.28291,-6844.607715,4.684865e+07
3633,1004082,1015322,4204,1,2,San Francisco,0.27655,2100.0,9282.80566,60.0,5.0,0.0,0.100460,59.999930,2567.13379,9282.80566,5035.59000,1670.32568,623.98383,1756.57861,98.16394,0.0000,0.0,0.0,0.00000,0.0,EV,10500.0,1.38275,5035.59000,1670.32568,623.98383,1756.57861,98.16394,Freeway GP,Urban Business,4204.0,EV,21161.263158,9184.64206,-11976.621098,1.434395e+08


In [52]:
comparison[comparison['facilityType']=='Arterial'].MODEL_LINK_ID.unique()

array([2104970, 2125215, 2272958, 3086539, 5070029], dtype=int64)

In [49]:
mtc_pems[mtc_pems['link_id']==21782]

,link_id,direction,type,time_period,count
70,21782.0,N,ML,AM,13025.125000
71,21782.0,N,ML,EA,1999.162791
72,21782.0,N,ML,EV,8977.128205
73,21782.0,N,ML,MD,13604.000000
74,21782.0,N,ML,PM,12296.250000


In [68]:
comparison.to_csv('../validation/Output/all_links.csv', index=False)

In [62]:
county_dict=['San Francisco',
             'San Mateo',
             'Santa Clara',
             'Alameda',
             'Contra Costa',
             'Solano',
             'Napa',
             'Sonoma',
             'Marin','San Joaquin']

county_sorter_index = dict(zip(county_dict, range(len(county_dict))))
summary=comparison.groupby(['facilityType',
                            'TIMEPERIOD']).agg(Number_of_Observation=('MODEL_LINK_ID','nunique'),
                                               Mean_Observed_Volume=('count','mean'),
                                               Mean_Model_Volume=('MODEL_VOL','mean'),
                                               MEAN_SQUARED_ERROR=('SQUARED_ERROR','mean')).reset_index()

summary['% RMSE'] =(summary['MEAN_SQUARED_ERROR']**0.5)/summary['Mean_Observed_Volume']
ft_sorter=['Freeway','Expressway','Arterial','Ramp','Collector','CentroidConnector']
tp_sorter=['24 Hour','EA','AM','MD','PM','EV']
ft_sorter_index = dict(zip(ft_sorter, range(len(ft_sorter))))
tp_sorter_index = dict(zip(tp_sorter, range(len(tp_sorter))))
summary=summary.sort_values(by='facilityType',key=lambda x: x.map(ft_sorter_index))
#.sort_values(by='TIMEPERIOD',key=lambda x: x.map(tp_sorter_index))
summary.to_csv('../validation/Output/summary_ft.csv', index=False)

In [34]:
county_dict=['San Francisco',
             'San Mateo',
             'Santa Clara',
             'Alameda',
             'Contra Costa',
             'Solano',
             'Napa',
             'Sonoma',
             'Marin']

county_sorter_index = dict(zip(county_dict, range(len(county_dict))))
summary=comparison[comparison['facilityType']=='Freeway'].groupby(['COUNTY','TIMEPERIOD']).agg(Number_of_Observation=('MODEL_LINK_ID','nunique'),
                                               Mean_Observed_Volume=('count','mean'),
                                               Mean_Model_Volume=('MODEL_VOL','mean'),
                                               MEAN_SQUARED_ERROR=('SQUARED_ERROR','mean')).reset_index()

summary['% RMSE'] =(summary['MEAN_SQUARED_ERROR']**0.5)/summary['Mean_Observed_Volume']
ft_sorter=['Freeway','Expressway','Arterial','Ramp','Collector','CentroidConnector']
tp_sorter=['24 Hour','EA','AM','MD','PM','EV']
ft_sorter_index = dict(zip(ft_sorter, range(len(ft_sorter))))
tp_sorter_index = dict(zip(tp_sorter, range(len(tp_sorter))))
summary=summary.sort_values(by='COUNTY',key=lambda x: x.map(county_sorter_index))

summary.to_csv('../validation/Output/validation_summary_by_county.csv', index=False)

In [42]:
summary.sort_values(by='COUNTY',key=lambda x: x.map(county_sorter_index))

,COUNTY,facilityType,TIMEPERIOD,Number_of_Observation,Mean_Observed_Volume,Mean_Model_Volume,MEAN_SQUARED_ERROR,% RMSE
59,San Francisco,Ramp,PM,2,9311.745690,290.677885,8.863637e+07,1.011055
55,San Francisco,Ramp,AM,2,9505.759470,548.252165,8.906445e+07,0.992808
54,San Francisco,Ramp,24 Hour,2,39941.437086,1113.886720,1.585836e+09,0.997024
58,San Francisco,Ramp,MD,2,10862.053571,165.331670,1.210588e+08,1.012946
53,San Francisco,Freeway,PM,21,21984.565434,20122.588626,5.335603e+07,0.332257
...,...,...,...,...,...,...,...,...
63,San Joaquin,Freeway,EV,3,6243.132414,11029.524150,1.325708e+08,1.844257
64,San Joaquin,Freeway,MD,3,7956.085359,11682.713013,1.409010e+08,1.491961
65,San Joaquin,Freeway,PM,3,9072.539668,14916.477463,5.605429e+07,0.825231
60,San Joaquin,Freeway,24 Hour,3,30141.065652,53470.613763,2.153162e+09,1.539500
